# Survey Plan AI - Cloud Trainer (Git-Ops)

This notebook pulls the latest code from GitHub and trains on your Drive data.

In [ ]:
# 1. Mount Google Drive (Data & Results)
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 2. Deploy Code (Git Clone/Pull)
import os
import shutil

REPO_URL = "https://github.com/14jadon14/survey-plan-AI"
REPO_NAME = REPO_URL.split("/")[-1].replace(".git", "")
CODE_DIR = f"/content/{REPO_NAME}"

if os.path.exists(CODE_DIR):
    print("🔄 Updating code...")
    !cd "{CODE_DIR}" && git pull
else:
    print("🚀 Cloning repository...")
    !git clone "{REPO_URL}"

print(f"✅ Code deployed to: {CODE_DIR}")

In [ ]:
# 3. Setup Environment & Data
# Install dependencies from the repo
!pip install -r "{CODE_DIR}/requirements.txt"

# Prepare Data (Fast IO: Copy zip from Drive -> Local VM)
# We assume data.zip is in 'My Drive/SurveyPlan AI/data.zip' OR just 'My Drive/data.zip'
DRIVE_ROOT = "/content/drive/MyDrive"
POSSIBLE_DATA_PATHS = [
    f"{DRIVE_ROOT}/SurveyPlan AI/data.zip",
    f"{DRIVE_ROOT}/data.zip"
]

DATA_ZIP = None
for p in POSSIBLE_DATA_PATHS:
    if os.path.exists(p):
        DATA_ZIP = p
        break

if DATA_ZIP:
    if not os.path.exists("/content/custom_data"):
        print(f"📦 Found data at {DATA_ZIP}. Unzipping to local VM...")
        !unzip -q -o "{DATA_ZIP}" -d /content/custom_data
    else:
        print("✅ Data already ready.")
else:
    print("⚠️ Warning: data.zip not found in Drive. Ensure it is in 'SurveyPlan AI' folder or Root.")

In [ ]:
# 4. Run Training
import sys

# Define where to save results (Back to Drive)
RESULTS_DIR = f"{DRIVE_ROOT}/SurveyPlan AI/runs"
if not os.path.exists(RESULTS_DIR):
    os.makedirs(RESULTS_DIR)

print(f"🚀 Starting Training... Results will be saved to: {RESULTS_DIR}")

# Run src/train.py
# Note: We don't rely on 'runs' inside the VM. We let YOLO save there, then copy back.
# Or we could modify config. But copying is safer for now.

script_path = f"{CODE_DIR}/src/train.py"

# We set PYTHONPATH so the script can 'import src...'
!PYTHONPATH="{CODE_DIR}" python "{script_path}" --data_path /content/custom_data

# 5. Backup Results
print("💾 Backing up runs to Drive...")
vm_runs = f"{CODE_DIR}/runs"
if os.path.exists(vm_runs):
    # Copy new runs to Drive
    !cp -r "{vm_runs}/." "{RESULTS_DIR}/"
    print("✅ Backup complete!")
else:
    print("⚠️ No runs folder found to backup.")